In [24]:
import numpy as np
import awkward as ak
np.random.seed(1337)  # for reproducibility

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.layers import GlobalAveragePooling3D,AveragePooling3D,ConvLSTM2D,TimeDistributed, Reshape, Input, Dense, Dropout, Flatten, Conv3D, MaxPooling3D,Conv2D, MaxPooling2D,BatchNormalization,AveragePooling2D, concatenate
from keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

from os import system
from os.path import exists

from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

import utils as pic

In [32]:
lr_init     = 1.e-3    # Initial learning rate  
batch_size  = 50       # Training batch size
train_size  = 2000     # Training size
valid_size  = 1000     # Validation size
test_size   = 1000     # Test size
epochs      = 5       # Number of epochs
doGPU       = False    # Use GPU
min_t       =-0.015 #Min time
max_t       =0.01 #Max time
t_step      =0.00099 #time step

In [33]:
# Set range of training set
train_start, train_stop = 0, train_size
assert train_stop > train_start
assert (len(pic.decays)*train_size) % batch_size == 0
X_train, y_train = pic.load_data(train_start,train_stop)

# Set range of validation set
valid_start, valid_stop = 160000, 160000+valid_size
assert valid_stop  >  valid_start
assert valid_start >= train_stop
X_valid, y_valid = pic.load_data(valid_start,valid_stop)

# Set range of test set
test_start, test_stop = 204800, 204800+test_size
assert test_stop  >  test_start
assert test_start >= valid_stop
X_test, y_test = pic.load_data(test_start,test_stop)

samples_requested = len(pic.decays) * (train_size + valid_size + test_size)
samples_available = len(y_train) + len(y_valid) + len(y_test)
assert samples_requested == samples_available

In [34]:
X_e_train,X_t_train,maxframes,t_bins,X_e_max_train,X_t_max_train = pic.timeordered_BC(X_train,cutoff=0.005,remove_empty=True,normalize=True,min_t = min_t,max_t = max_t,t_step=t_step)
y_b_train = to_categorical(y_train)
X_e_valid,X_t_valid,_,t_bins,X_e_max_valid,X_t_max_valid = pic.timeordered_BC(X_valid,cutoff=0.005,remove_empty=True,normalize=True,min_t = min_t,max_t = max_t,t_step=t_step)
y_b_valid = to_categorical(y_valid)
X_e_test,X_t_test,_,t_bins,X_e_max_test,X_t_max_test = pic.timeordered_BC(X_test,cutoff=0.005,remove_empty=True,normalize=True,min_t = min_t,max_t = max_t,t_step=t_step)
y_b_test = to_categorical(y_test)

In [35]:
#Input image
input_3d = Input(shape=(maxframes,32, 32, 1))

In [36]:
input_3d.shape

TensorShape([None, 26, 32, 32, 1])

In [37]:
#MRI scan conv3d 
model_name = 'cnn_3d_1'

x = Conv3D(filters=64, kernel_size=3,padding='same', activation='relu')(input_3d)
x = MaxPooling3D(pool_size=2)(x)
x = BatchNormalization()(x)

x = Conv3D(filters=64, kernel_size=3,padding='same',activation='relu')(x)
x = MaxPooling3D(2)(x)
x = BatchNormalization()(x)

x = Conv3D(filters=128, kernel_size=3,padding='same', activation='relu')(x)
x = MaxPooling3D(2)(x)
x = BatchNormalization()(x)

x = GlobalAveragePooling3D()(x)
#x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(1, activation='sigmoid', kernel_initializer='TruncatedNormal')(x)
model_1 = Model([input_3d],output,name='cnn_3d_1')
#model.summary()

In [38]:
plot_model(model_1,show_shapes=True,to_file=f'{model_name}.png')
system('mv *.png Models/');
#Checkpoint and reduce lr
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1.e-6)
checkpoint_cb = ModelCheckpoint("conv3d_1.h5", save_best_only=True)

#Compile
model_1.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr_init),metrics=['accuracy'])

In [39]:
#RCNN 3D
model_name = 'rcnn_3d_1'

x = ConvLSTM2D(filters=32, kernel_size=3,padding='same', activation='relu',return_sequences=True)(input_3d)
x = AveragePooling3D(pool_size=(1,2,2))(x)
x = BatchNormalization()(x)

x = ConvLSTM2D(filters=64, kernel_size=2,padding='same',activation='relu',return_sequences=True)(x)
x = AveragePooling3D(pool_size=(1,2,2))(x)
x = BatchNormalization()(x)

x = Conv3D(filters=128, kernel_size=2,padding='same', activation='relu')(x)
x = MaxPooling3D(2)(x)
x = BatchNormalization()(x)

x = Conv3D(filters=256, kernel_size=1,padding='same', activation='relu')(x)
x = MaxPooling3D(2)(x)
x = BatchNormalization()(x)

x = GlobalAveragePooling3D()(x)
#x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(1, activation='sigmoid', kernel_initializer='TruncatedNormal')(x)
model_2 = Model([input_3d],output,name=model_name)
model_2.summary()

Model: "rcnn_3d_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 26, 32, 32, 1)]   0         
_________________________________________________________________
conv_lst_m2d_8 (ConvLSTM2D)  (None, 26, 32, 32, 32)    38144     
_________________________________________________________________
average_pooling3d_8 (Average (None, 26, 16, 16, 32)    0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 26, 16, 16, 32)    128       
_________________________________________________________________
conv_lst_m2d_9 (ConvLSTM2D)  (None, 26, 16, 16, 64)    98560     
_________________________________________________________________
average_pooling3d_9 (Average (None, 26, 8, 8, 64)      0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 26, 8, 8, 64)      25

In [40]:
plot_model(model_2,show_shapes=True,to_file=f'{model_name}.png')
system('mv *.png Models/');
#Checkpoint and reduce lr
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1.e-6)
checkpoint_cb = ModelCheckpoint("conv3d_1.h5", save_best_only=True)

#Compile
model_2.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr_init),metrics=['accuracy'])

In [19]:
history_1 = model_1.fit(
    X_e_train, y_train,
    validation_data=(X_e_valid,y_valid),
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    callbacks=[checkpoint_cb,reduce_lr],
    verbose=1
)

8/8 [==============================] - 172s 23s/step - loss: 0.6974 - accuracy: 0.5975 - val_loss: 0.7221 - val_accuracy: 0.5000


In [ ]:
history_2 = model_2.fit(
    X_e_train, y_train,
    validation_data=(X_e_valid,y_valid),
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    callbacks=[checkpoint_cb,reduce_lr],
    verbose=1
)

In [20]:
#Prepare to plot stuff
pic.plot_stuff()

In [26]:
pic.plot_history(history,metric='loss',save=True,fname=f'history_{model_name}.png')

TypeError: plot_history() got an unexpected keyword argument 'fname'